In [2]:
# module
import numpy as np
import pandas as pd
import tqdm
from haversine import haversine
from tqdm import tqdm_pandas, tqdm

In [6]:
#load data
cw = pd.read_csv('횡단보도_완료.csv')
accademy = pd.read_csv('강서학원.csv')
accademy2 = pd.read_csv('강서학원2.csv')

In [17]:
acca = pd.concat([accademy.dropna(), accademy2], axis = 0).reset_index().iloc[:,1:]

In [19]:
acca = acca[['학원명', 'Latitude', 'Longitude']] 

In [20]:
# 차 사고
cw.columns = ['crosswalk_id', 'cw_lon', 'cw_lat']

# 횡단보도
acca.columns = ['accademy', 'ac_lat', 'ac_lon']

In [21]:
# key 만들기
cw['key'] = 0
acca['key'] = 0

C:\Users\abc\anaconda3\envs\please\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
# 데이터 합치기
prod = pd.merge(cw, acca, on = 'key', how = 'outer')

In [23]:
## 하버사인 거리
lat1 = np.deg2rad(prod['cw_lat'])
lat2 = np.deg2rad(prod['ac_lat'])
lon1 = np.deg2rad(prod['cw_lon'])
lon2 = np.deg2rad(prod['ac_lon'])
dlat = lat2 - lat1 
dlon = lon2 - lon1 
a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
c = 2 * np.arcsin(np.sqrt(a))

In [24]:
# 미터거리 구하기
prod['distance'] = c * 6371 * 1000

In [25]:
# 상세한 확인
prod['distance'].describe()

count    3.362634e+06
mean     2.522955e+03
std      1.358845e+03
min      3.469479e+00
25%      1.491314e+03
50%      2.380174e+03
75%      3.345092e+03
max      8.714040e+03
Name: distance, dtype: float64

In [26]:
p100 = prod[prod['distance'] <= 100]
cw_acca_100 = p100.groupby('crosswalk_id').count().reset_index().iloc[:,:2]
cw_acca_100.columns = ['crosswalk_id', 'acca_count']
cw_acca_100.to_csv('횡단보도_100미터_학원.csv', index = False, encoding = 'cp949')
cw_acca_100.describe()

,acca_count
count,1077.000000
mean,7.935933
std,14.349157
min,1.000000
25%,1.000000
50%,3.000000
75%,8.000000
max,108.000000


In [27]:
p50 = prod[prod['distance'] <= 50]
cw_acca_50 = p50.groupby('crosswalk_id').count().reset_index().iloc[:,:2]
cw_acca_50.columns = ['crosswalk_id', 'acca_count']
cw_acca_50.to_csv('횡단보도_50미터_학원.csv', index = False, encoding = 'cp949')
cw_acca_50.describe()

,acca_count
count,544.000000
mean,4.426471
std,7.028626
min,1.000000
25%,1.000000
50%,2.000000
75%,5.000000
max,71.000000
